<a href="https://colab.research.google.com/github/JONNY-ME/AIBot-telegram/blob/master/Anglophone2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install catboost -q
from google.colab import drive
drive.mount('/content/drive')

     |████████████████████████████████| 76.6 MB 1.2 MB/s 
Mounted at /content/drive


In [2]:
import os
import pandas as pd
import warnings
from catboost import CatBoostClassifier
from itertools import combinations
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import BaggingClassifier



path = "/content/drive/MyDrive/Anglophpne/"
seed = 21
le = LabelEncoder()
os.chdir(path)
warnings.filterwarnings('ignore')

In [3]:
Train = pd.read_csv("Train.csv")
Test = pd.read_csv("Test.csv")

In [4]:
Train["is_train"] = 1
Test["is_train"] = 0

train_test = pd.concat([Train, Test])
train_test.shape

(24675, 22)

In [5]:
cat_cols = ["age_group", "travel_with", "purpose", "main_activity", "info_source", "tour_arrangement", "first_trip_tz"]

train_test["main_activity"].loc[train_test.main_activity == "Widife Tourism"] = "Wildlife Tourism"
train_test["total_person"] = train_test["total_female"] + train_test["total_male"]
train_test["total_night"] = train_test["night_mainland"] + train_test["night_zanzibar"]


for col in cat_cols:
  train_test[col] = le.fit_transform(train_test[col])

train_test["country"] = le.fit_transform(train_test["country"])
county_mapping = dict(zip(le.classes_, range(len(le.classes_))))

packages = ["package_transport_int", "package_accomodation", "package_food", "package_transport_tz", "package_sightseeing",
            "package_guided_tour", "package_insurance"]
for package in packages:
  train_test[package] = train_test[package].apply(lambda x:["No", "Yes"].index(x))



In [6]:
train = train_test[train_test["is_train"] == 1].drop("is_train", axis=1)
train.fillna(0, inplace=True)

In [7]:
cost_map1 = {
    'Lower Cost': 0,
    'Low Cost': 0,
    'Normal Cost': 1,
    'High Cost': 2, 
    'Higher Cost': 2, 
    'Highest Cost': 2,
}
cost_map2 = {
    'Lower Cost' : 0,
    'Low Cost' : 1,
    'Normal Cost': 0,
    'High Cost' : 0,
    'Higher Cost' : 1,
    'Highest Cost' : 2,
}

train["category"] = train["cost_category"].apply(cost_map1.get)
train["spec_category"] = train["cost_category"].apply(cost_map2.get)

In [8]:
train

,Tour_ID,country,age_group,travel_with,total_female,total_male,purpose,main_activity,info_source,tour_arrangement,...,package_guided_tour,package_insurance,night_mainland,night_zanzibar,first_trip_tz,cost_category,total_person,total_night,category,spec_category
0,tour_id1hffseyw,62,2,1,0.0,2.0,6,0,0,1,...,0,0,0,7,1,High Cost,2.0,7,2,0
1,tour_idnacd7zag,133,1,3,1.0,1.0,1,9,7,1,...,0,0,0,7,1,High Cost,2.0,7,2,0
2,tour_id62vz7e71,134,3,3,1.0,1.0,1,8,7,1,...,1,0,6,6,1,Higher Cost,2.0,12,2,1
3,tour_idrc76tzix,106,1,4,3.0,1.0,1,0,4,0,...,0,0,3,0,0,Lower Cost,4.0,3,0,0
4,tour_idn723m0n9,134,2,0,0.0,1.0,1,8,7,1,...,1,1,7,0,1,Higher Cost,1.0,7,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18501,tour_idmp5ciw50,68,2,0,0.0,1.0,4,6,3,0,...,0,0,3,0,0,Lower Cost,1.0,3,0,0
18502,tour_ideq0yncfs,68,2,0,1.0,0.0,3,9,3,0,...,0,0,2,0,0,Lower Cost,1.0,2,0,0
18503,tour_idv7pz3vs8,134,1,4,2.0,1.0,1,8,7,0,...,0,0,9,0,1,Higher Cost,3.0,9,2,1
18504,tour_idy6ydo00w,134,1,3,1.0,1.0,1,3,4,1,...,1,0,13,4,0,Higher Cost,2.0,17,2,1


In [9]:
X = train.drop(["Tour_ID", "cost_category", 'category', 'spec_category'], axis=1)
y = train[['category', "cost_category", "spec_category"]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21)

In [10]:

rnf_params = {'max_depth': 16, 'max_features':'log2', 'n_estimators': 1000, 'random_state': seed}
lgb_prams = {"learning_rate":0.009, "n_estimators":1000, "objective":'multiclass', 'random_state':seed}
xgb_params = {"eta":0.3, 'n_estimators': 200, 'random_state':seed}
cat_params = {"iterations":1000, "learning_rate":0.01, "verbose":0, "random_state":seed, }

def model():
  models = [
            ('lgb', LGBMClassifier(**lgb_prams)),
            ('rnd', RandomForestClassifier(**rnf_params)),
            ('xgb', XGBClassifier(**xgb_params)),
            ('cat', CatBoostClassifier(**cat_params)),
  ]

  return VotingClassifier(estimators=models, voting='soft', flatten_transform=True)

  
def model2():
  models = [
            # ('lgb', LGBMClassifier(**lgb_prams)),
            ('rnd', RandomForestClassifier(**rnf_params)),
            ('xgb', XGBClassifier(**xgb_params)),
            ('cat', CatBoostClassifier(**cat_params)),
  ]

  return VotingClassifier(estimators=models, voting='soft', flatten_transform=True)

main_model = model()

main_model.fit(X_train.values, y_train.category)

VotingClassifier(estimators=[('lgb',
                              LGBMClassifier(learning_rate=0.009,
                                             n_estimators=1000,
                                             objective='multiclass',
                                             random_state=21)),
                             ('rnd',
                              RandomForestClassifier(max_depth=16,
                                                     max_features='log2',
                                                     n_estimators=1000,
                                                     random_state=21)),
                             ('xgb',
                              XGBClassifier(eta=0.3, n_estimators=200,
                                            random_state=21)),
                             ('cat',
                              <catboost.core.CatBoostClassifier object at 0x7f1dce1659d0>)],
                 voting='soft')

In [11]:
pred = main_model.predict_proba(X_test.values)
pred_df = pd.DataFrame(pred, columns=["Low", "Medium", "High"], index=X_test.index)
log_loss(y_test.category, pred)

0.6330306691213579

In [15]:
preds = {}
pred_df['mlower'] = 1/2
pred_df['mlow'] = 1/2
pred_df['mnorm'] = 1
pred_df['mhigh'] = 1/3
pred_df['mhigher'] = 1/3
pred_df['mhighest'] = 1/3

cols = [['mlower', 'mlow'], ['mnorm'], ['mhigh', 'mhigher', 'mhighest']]
for cat in [0, 2]:
  inx1 = y_train[y_train.category == cat].index
  xtr = X_train.loc[inx1]
  ytr = y_train.loc[inx1]


  clf = model2()
  clf.fit(xtr, ytr.spec_category)
  pred = clf.predict_proba(X_test)
  for i in range(pred.shape[1]):
    pred_df[cols[cat][i]] = pred[:, i]


pred_df


,Low,Medium,High,mlower,mlow,mnorm,mhigh,mhigher,mhighest,mapper
7604,0.509919,0.445207,0.044874,0.612720,0.387280,1,0.786890,0.211291,0.001820,0
3169,0.026477,0.110416,0.863106,0.631553,0.368447,1,0.447683,0.541756,0.010561,1
17535,0.035904,0.066172,0.897923,0.760146,0.239854,1,0.112133,0.849918,0.037949,2
7628,0.637779,0.332235,0.029987,0.604130,0.395870,1,0.872412,0.123794,0.003795,0
9973,0.039138,0.279744,0.681118,0.829814,0.170186,1,0.569947,0.420296,0.009758,2
...,...,...,...,...,...,...,...,...,...,...
12674,0.808124,0.182260,0.009615,0.824038,0.175962,1,0.755015,0.242744,0.002241,0
10066,0.019892,0.048147,0.931961,0.728466,0.271534,1,0.236930,0.533054,0.230016,2
3099,0.012024,0.049372,0.938604,0.275314,0.724686,1,0.316391,0.675848,0.007761,2
5999,0.352216,0.595772,0.052012,0.472363,0.527637,1,0.768560,0.229294,0.002146,1


In [16]:
pred_df['Lower Cost'] = pred_df['Low'] * pred_df['mlower']
pred_df['Low Cost'] = pred_df['Low'] * pred_df['mlow']
pred_df['Normal Cost'] = pred_df['Medium'] * pred_df['mnorm']
pred_df['High Cost'] = pred_df['High'] * pred_df['mhigh']
pred_df['Higher Cost'] = pred_df['High'] * pred_df['mhigher']
pred_df['Highest Cost'] = pred_df['High'] * pred_df['mhighest']

cost_map = {
    'Lower Cost' : 0,
    'Low Cost' : 1,
    'Normal Cost' : 2,
    'High Cost' : 3,
    'Higher Cost' : 4,
    'Highest Cost' : 5
}

_pred = pred_df[list(cost_map.keys())]
log_loss(y_test.cost_category.apply(cost_map.get), _pred)

1.0882874417202508

In [19]:
test = train_test[train_test["is_train"]==0].drop(["cost_category", "is_train"], axis=1)
test.fillna(0, inplace=True)
test

,Tour_ID,country,age_group,travel_with,total_female,total_male,purpose,main_activity,info_source,tour_arrangement,...,package_food,package_transport_tz,package_sightseeing,package_guided_tour,package_insurance,night_mainland,night_zanzibar,first_trip_tz,total_person,total_night
0,tour_idynufedne,69,1,0,0.0,1.0,1,8,3,0,...,0,0,0,0,0,7,4,1,1.0,11
1,tour_id9r3y5moe,133,2,1,1.0,1.0,1,3,7,1,...,1,1,1,1,1,7,0,1,2.0,7
2,tour_idf6itml6g,62,1,3,1.0,1.0,1,0,7,1,...,0,1,0,0,0,0,6,1,2.0,6
3,tour_id99u4znru,68,1,0,0.0,1.0,4,0,4,0,...,0,0,0,0,0,3,4,0,1.0,7
4,tour_idj4i9urbx,139,1,0,0.0,1.0,0,8,4,0,...,0,0,0,0,0,6,0,0,1.0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6164,tour_id2deyfjhq,140,1,0,0.0,1.0,0,9,0,0,...,0,0,0,0,0,2,0,0,1.0,2
6165,tour_idlenv2rio,38,1,0,0.0,1.0,6,6,0,0,...,0,0,0,0,0,60,0,1,1.0,60
6166,tour_id7wwqrs0p,23,1,0,0.0,1.0,1,0,0,1,...,1,1,0,0,0,5,0,0,1.0,5
6167,tour_idx80vbw5a,23,0,0,1.0,0.0,6,9,0,0,...,0,0,0,0,0,21,0,0,1.0,21


In [21]:
main_model.fit(X.values, y.category)
pred = main_model.predict_proba(test.drop("Tour_ID", axis=1).values)
pred_df = pd.DataFrame(pred, columns=["Low", "Medium", "High"], index=test.index)

In [23]:
preds = {}
pred_df['mlower'] = 1/2
pred_df['mlow'] = 1/2
pred_df['mnorm'] = 1
pred_df['mhigh'] = 1/3
pred_df['mhigher'] = 1/3
pred_df['mhighest'] = 1/3

cols = [['mlower', 'mlow'], ['mnorm'], ['mhigh', 'mhigher', 'mhighest']]
for cat in [0, 2]:
  inx1 = y[y.category == cat].index
  xtr = X.loc[inx1]
  ytr = y.loc[inx1]


  clf = model2()
  clf.fit(xtr, ytr.spec_category)
  pred = clf.predict_proba(test.drop("Tour_ID", axis=1))
  for i in range(pred.shape[1]):
    pred_df[cols[cat][i]] = pred[:, i]


pred_df

,Low,Medium,High,mlower,mlow,mnorm,mhigh,mhigher,mhighest
0,0.117670,0.726394,0.155936,0.346878,0.653122,1,0.812644,0.185544,0.001812
1,0.021070,0.106650,0.872280,0.589969,0.410031,1,0.385986,0.573694,0.040320
2,0.016430,0.129508,0.854062,0.224449,0.775551,1,0.785714,0.205109,0.009177
3,0.309421,0.615588,0.074991,0.256105,0.743895,1,0.746005,0.252325,0.001670
4,0.597780,0.371657,0.030563,0.568131,0.431869,1,0.786050,0.212227,0.001723
...,...,...,...,...,...,...,...,...,...
6164,0.791392,0.189072,0.019536,0.638518,0.361482,1,0.841901,0.155554,0.002545
6165,0.405769,0.430216,0.164015,0.513078,0.486922,1,0.682828,0.314779,0.002393
6166,0.047392,0.450690,0.501918,0.501766,0.498234,1,0.673890,0.317698,0.008413
6167,0.330804,0.573466,0.095731,0.503980,0.496020,1,0.842576,0.155625,0.001799


In [30]:
pred_df['Lower Cost'] = pred_df['Low'] * pred_df['mlower']
pred_df['Low Cost'] = pred_df['Low'] * pred_df['mlow']
pred_df['Normal Cost'] = pred_df['Medium'] * pred_df['mnorm']
pred_df['High Cost'] = pred_df['High'] * pred_df['mhigh']
pred_df['Higher Cost'] = pred_df['High'] * pred_df['mhigher']
pred_df['Highest Cost'] = pred_df['High'] * pred_df['mhighest']

cost_map = {
    'Lower Cost' : 0,
    'Low Cost' : 1,
    'Normal Cost' : 2,
    'High Cost' : 3,
    'Higher Cost' : 4,
    'Highest Cost' : 5
}

pred_df['Tour_ID'] = test.Tour_ID
sub = pred_df[['Tour_ID', 'High Cost', 'Higher Cost', 'Highest Cost', 'Low Cost', 'Lower Cost', 'Normal Cost']]
sub.head()

,Tour_ID,High Cost,Higher Cost,Highest Cost,Low Cost,Lower Cost,Normal Cost
0,tour_idynufedne,0.126720,0.028933,0.000283,0.076853,0.040817,0.726394
1,tour_id9r3y5moe,0.336688,0.500422,0.035170,0.008639,0.012431,0.106650
2,tour_idf6itml6g,0.671049,0.175175,0.007838,0.012742,0.003688,0.129508
3,tour_id99u4znru,0.055944,0.018922,0.000125,0.230176,0.079244,0.615588
4,tour_idj4i9urbx,0.024024,0.006486,0.000053,0.258163,0.339617,0.371657


In [31]:
sub.to_csv('/content/half-day.csv', index=False)